# Create Daily Heists

In [1]:
import os, sys, random, json
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [5]:
import django.db.models as JM
from django.contrib.auth import get_user_model
from django.utils import timezone

In [6]:
import emporium.models as EM
import emporium.logic.guild as GD
import emporium.logic.stage as ST
import emporium.logic.simulation as SM
print([x for x in dir(EM) if x[0]!='_' and len(x)>8])

['FactoryUpgrades', 'GothicTower', 'KeepUpgrades', 'LeagueTrial', 'SingletonUpgrades', 'StorageUpgrades', 'ThiefLevel', 'UnlockableItem', 'UnlockableThief']


In [7]:
import engine.models as GM
import engine.logic.resource as RS
import engine.logic.content as CT
import engine.logic.launcher as LH
import engine.logic.robot as RB
print([x for x in dir(GM) if x[0]!='_' and len(x)>8])

['CommonWares', 'GuildAssets', 'GuildRamp', 'GuildTower', 'ItemInGuild', 'ItemUnlocked', 'MagicMarket', 'ThiefInGuild', 'ThiefUnlocked']


In [8]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Daily Heists

In [9]:
# get user from request

userMd = get_user_model().objects.filter(user_name='Admin')[0]
userMd

<User: admin@thieves-guild.com>

In [10]:
guildMd = GM.Guild.objects.GetOrNone(UserFK=userMd, Selected=True)

if not guildMd:
    responseDx = {
        'thiefLs': None,
        'assetLs': None,
        'message': '* A guild must be chosen in the Account page.',
    }

guildMd

<Guild: Guild object (10)>

In [11]:
currDt = timezone.now()
currMonth = currDt.month
currDay = currDt.day
currWeekDay = currDt.weekday()   # 0 monday, 1 tuesday, .. 6 sunday
currDate = f"{currDt.year}-{str(currMonth).zfill(2)}-{str(currDay).zfill(2)}"

print('month:', currMonth, '| day:', currDay, '| week day:', currWeekDay, '| full: ', currDate)

month: 1 | day: 5 | week day: 4 | full:  2024-01-05


In [16]:
GM.GuildTower.objects.all().delete()

(24, {'engine.GuildTower': 24})

In [13]:
# guild tower

towerLs = CT.GetOr CreateTower(guildMd, currDate)
len(towerLs)

SyntaxError: invalid syntax (1952610833.py, line 3)

In [ ]:
towerLs = GM.GuildTower.objects.filter(GuildFK=guildMd).values()
PD.DataFrame(tower).drop(['_state', 'GuildFK_id'], axis=1, errors='ignore')

In [24]:
tower = GM.GuildTower.objects.filter(GuildFK=guildMd).values()
len(tower)
towerDf = PD.DataFrame(tower).drop(['_state', 'GuildFK_id'], axis=1, errors='ignore')
towerDf = towerDf.drop_duplicates(subset=['StageNo']).sort_values('StageNo')
towerDf

24

,id,StageNo,CreateDate,TypeR1,TypeR2,TypeR3,ObstaclesR1,ObstaclesR2,ObstaclesR3,CompleteR1,CompleteR2,CompleteR3
0,301,1,2024-01-05,balanced,None,None,"[{""Level"": 1, ""Name"": ""Warden"", ""Trait"": ""All""...",None,None,False,False,False
1,302,2,2024-01-05,biased mig,None,None,"[{""Level"": 1, ""Name"": ""Chandelier"", ""Trait"": ""...",None,None,False,False,False
2,303,3,2024-01-05,balanced,None,None,"[{""Level"": 1, ""Name"": ""Barrel"", ""Trait"": ""Mig""...",None,None,False,False,False
3,304,4,2024-01-05,biased agi,None,None,"[{""Level"": 1, ""Name"": ""Arcane Seal"", ""Trait"": ...",None,None,False,False,False
4,305,5,2024-01-05,balanced,None,None,"[{""Level"": 1, ""Name"": ""Door"", ""Trait"": ""Agi"", ...",None,None,False,False,False
8,309,6,2024-01-05,balanced,balanced,None,"[{""Level"": 2, ""Name"": ""Barrel"", ""Trait"": ""Mig""...","[{""Level"": 1, ""Name"": ""Arcane Seal"", ""Trait"": ...",None,False,False,False
9,310,7,2024-01-05,biased cun,balanced,None,"[{""Level"": 1, ""Name"": ""Spike Trap"", ""Trait"": ""...","[{""Level"": 1, ""Name"": ""Spike Trap"", ""Trait"": ""...",None,False,False,False
10,311,8,2024-01-05,balanced,balanced,None,"[{""Level"": 1, ""Name"": ""Crossbow Trap"", ""Trait""...","[{""Level"": 2, ""Name"": ""Vanguard"", ""Trait"": ""Al...",None,False,False,False
11,312,9,2024-01-05,biased cmb,balanced,None,"[{""Level"": 2, ""Name"": ""Gargoyle"", ""Trait"": ""Mi...","[{""Level"": 2, ""Name"": ""Gargoyle"", ""Trait"": ""Mi...",None,False,False,False
16,317,10,2024-01-05,balanced,balanced,biased agi,"[{""Level"": 1, ""Name"": ""Chandelier"", ""Trait"": ""...","[{""Level"": 1, ""Name"": ""Chandelier"", ""Trait"": ""...","[{""Level"": 2, ""Name"": ""Warden"", ""Trait"": ""All""...",False,False,False


In [ ]:
# league trials

